In [8]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
import random
import math
import csv
import os

# ---------- GLOBAL VARIABEL & KONSTANTA ----------
canvas_height = 0
canvas_width = 0
active_nodes = []
total_energy = []
avg_energy = []
throughputs = []

window_width = 1000

AREA_WIDTH = 100
AREA_HEIGHT = 100
BASE_STATION = (AREA_WIDTH / 2, AREA_HEIGHT / 2)
DATA_PACKET_SIZE = 4000
EDA = 5e-9
E_FS = 10e-12
E_ELEC = 50e-9

NUM_NODES = 0
ROUNDS = 0
INITIAL_ENERGY = 0

nodes = []

# Simpan referensi gambar untuk canvas
canvas_topology_images = []

# --------- LEACH & SIMULASI FUNCTIONS ----------
def calculate_tx_energy(packet_size, distance):
    return E_ELEC * packet_size + E_FS * packet_size * distance**2

def calculate_rx_energy(packet_size):
    return E_ELEC * packet_size

def calculate_distance_to_sink(node):
    return math.sqrt((node.x - BASE_STATION[0])**2 + (node.y - BASE_STATION[1])**2)

class Node:
    def __init__(self, node_id, x, y, energy=2.0):
        self.id = node_id
        self.x = x
        self.y = y
        self.energy = energy
        self.is_ch = False
        self.cluster_id = -1
        self.alive = True
        self.distance_to_sink = calculate_distance_to_sink(self)
        self.luminous_intensity = energy  # Cahaya berdasarkan energi node
        self.radius = 10  # Radius pencarian untuk GSO
        self.pheromone = 0  # Pheromone yang dimiliki node

fnd_achieved = False
hnd_achieved = False
lnd_achieved = False
half_nodes_dead = False
fnd_round = None
hnd_round = None
lnd_round = None

def reset_death_metrics():
    global fnd_achieved, hnd_achieved, lnd_achieved, half_nodes_dead
    global fnd_round, hnd_round, lnd_round
    fnd_achieved = False
    hnd_achieved = False
    lnd_achieved = False
    half_nodes_dead = False
    fnd_round = None
    hnd_round = None
    lnd_round = None
    label_fnd.config(text="FND: Not achieved")
    label_hnd.config(text="HND: Not achieved")
    label_lnd.config(text="LND: Not achieved")

def check_death_metrics(round_num):
    global fnd_achieved, hnd_achieved, lnd_achieved, half_nodes_dead, fnd_round, hnd_round, lnd_round
    dead_nodes = [node for node in nodes if not node.alive]
    num_dead_nodes = len(dead_nodes)
    if not fnd_achieved and num_dead_nodes >= 1:
        fnd_achieved = True
        fnd_round = round_num
        update_fnd_status(fnd_round)
    if not hnd_achieved and num_dead_nodes >= NUM_NODES / 2 and not half_nodes_dead:
        hnd_achieved = True
        half_nodes_dead = True
        hnd_round = round_num
        update_hnd_status(hnd_round)
    if not lnd_achieved and num_dead_nodes >= NUM_NODES:
        lnd_achieved = True
        lnd_round = round_num
        update_lnd_status(lnd_round)

def update_fnd_status(round_num):
    if fnd_achieved:
        label_fnd.config(text=f"FND tercapai pada ronde {round_num}.")

def update_hnd_status(round_num):
    if hnd_achieved:
        label_hnd.config(text=f"HND tercapai pada ronde {round_num}.")

def update_lnd_status(round_num):
    if lnd_achieved:
        label_lnd.config(text=f"LND tercapai pada ronde {round_num}.")

def update_glowworm_lighting_and_pheromone():
    for node in nodes:
        if node.alive:
            # Update cahaya (luminous intensity) berdasarkan energi
            node.luminous_intensity = node.energy
            
            # Update pheromone berdasarkan jarak ke cluster head
            if node.is_ch:
                node.pheromone = 1  # Cluster head memiliki pheromone maksimum
            else:
                nearest_ch = find_nearest_cluster_head(node)
                if nearest_ch:
                    dist = calculate_distance_to_sink(node)
                    node.pheromone = 1 / (1 + dist)  # Pheromone berbanding terbalik dengan jarak

def find_nearest_cluster_head(node):
    # Cari cluster head terdekat untuk node tertentu
    ch_nodes = [n for n in nodes if n.is_ch and n.alive]
    if ch_nodes:
        return min(ch_nodes, key=lambda ch: calculate_distance_to_sink(node))
    return None

def select_cluster_heads_gso(round_num, p=0.1):
    alive_nodes = [node for node in nodes if node.alive]
    if not alive_nodes:
        return
    
    for node in alive_nodes:
        node.is_ch = False  # Reset status cluster head

    # Tentukan cluster head berdasarkan cahaya dan pheromone terbaik
    for node in alive_nodes:
        best_cluster_head = None
        best_luminous_intensity = -float('inf')
        for other_node in alive_nodes:
            if other_node.luminous_intensity > best_luminous_intensity and other_node != node:
                best_luminous_intensity = other_node.luminous_intensity
                best_cluster_head = other_node
        
        # Jika cahaya cukup besar, node menjadi cluster head
        if best_cluster_head and random.random() < p:
            node.is_ch = True
            node.cluster_id = best_cluster_head.id

def form_clusters():
    for node in nodes:
        if not node.alive or node.is_ch:
            continue
        min_dist = float('inf')
        closest_ch = None
        for ch_node in nodes:
            if ch_node.alive and ch_node.is_ch:
                dist = math.sqrt((node.x - ch_node.x)**2 + (node.y - ch_node.y)**2)
                if dist < min_dist:
                    min_dist = dist
                    closest_ch = ch_node
        if closest_ch:
            node.cluster_id = closest_ch.id

def transmit_data():
    throughput = 0
    for node in nodes:
        if not node.alive:
            continue
        if not node.is_ch and node.cluster_id != -1:
            ch_node = next((n for n in nodes if n.id == node.cluster_id), None)
            if ch_node and ch_node.alive:
                dist = math.sqrt((node.x - ch_node.x)**2 + (node.y - ch_node.y)**2)
                node.energy -= calculate_tx_energy(DATA_PACKET_SIZE, dist)
                ch_node.energy -= calculate_rx_energy(DATA_PACKET_SIZE)
        elif node.is_ch and node.alive:
            node.energy -= EDA * DATA_PACKET_SIZE
            node.energy -= calculate_tx_energy(DATA_PACKET_SIZE, node.distance_to_sink)
            throughput += DATA_PACKET_SIZE
    return throughput

def check_energy():
    for node in nodes:
        if node.energy <= 0:
            node.alive = False

def log_round(round_num):
    header = ["Round", "Node ID", "X", "Y", "Energy", "Is CH", "Cluster ID", "Alive"]
    file_exists = os.path.isfile('leach_log.csv')
    with open('leach_log.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists or os.path.getsize('leach_log.csv') == 0:
            writer.writerow(header)
        for node in nodes:
            writer.writerow([round_num, node.id, node.x, node.y, max(node.energy, 0), int(node.is_ch), node.cluster_id, int(node.alive)])

# ----------- VISUALISASI (pakai dua canvas/tab) -----------  
def visualize_network(round_num):
    if round_num % 100 == 0 or round_num == ROUNDS - 1:
        plt.figure(figsize=(7, 7))
        non_ch = [node for node in nodes if node.alive and not node.is_ch]
        plt.scatter([node.x for node in non_ch], [node.y for node in non_ch], c='blue', label='Node')
        ch_nodes = [node for node in nodes if node.alive and node.is_ch]
        plt.scatter([node.x for node in ch_nodes], [node.y for node in ch_nodes], c='red', s=100, label='Cluster Head')
        plt.scatter([BASE_STATION[0]], [BASE_STATION[1]], c='green', s=200, marker='*', label='Sink')
        for node in non_ch:
            if node.cluster_id != -1:
                ch = next((n for n in ch_nodes if n.id == node.cluster_id), None)
                if ch:
                    plt.plot([node.x, ch.x], [node.y, ch.y], 'gray', alpha=0.3)
        for ch in ch_nodes:
            plt.plot([ch.x, BASE_STATION[0]], [ch.y, BASE_STATION[1]], 'black', alpha=0.5)
        plt.title(f'Topologi Jaringan - Round {round_num}')
        plt.xlabel('X (m)')
        plt.ylabel('Y (m)')
        plt.legend()
        plt.grid(True)
        image_path = f'topology_{round_num}.png'
        plt.savefig(image_path)
        plt.close()

def show_all_topology_images():
    canvas_topology.delete("all")
    y_offset = 10
    x_offset = 10
    images_per_row = 3    # Ubah ke 3 jika ingin 3 per baris
    img_width = 400
    img_height = 400
    padding_x = 20
    padding_y = 20

    canvas_topology_images.clear()
    img_count = 0
    for i in range(0, ROUNDS, 100):
        img_path = f"topology_{i}.png"
        if os.path.exists(img_path):
            img = Image.open(img_path)
            img = img.resize((img_width, img_height), Image.Resampling.LANCZOS)
            img_tk = ImageTk.PhotoImage(img)
            # Store all images in list to prevent garbage collection
            canvas_topology_images.append(img_tk)
            col = img_count % images_per_row
            row = img_count // images_per_row
            xpos = x_offset + col * (img_width + padding_x)
            ypos = y_offset + row * (img_height + padding_y)
            canvas_topology.create_image(xpos, ypos, anchor="nw", image=img_tk)
            img_count += 1

    total_rows = (img_count + images_per_row - 1) // images_per_row
    canvas_topology.config(
        scrollregion=(0, 0, 
                      x_offset + images_per_row * (img_width + padding_x),
                      y_offset + total_rows * (img_height + padding_y))
    )

def visualize_performance():
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    axs[0, 0].plot(range(ROUNDS), active_nodes)
    axs[0, 0].set_title('Node Active vs Rounds')
    axs[0, 1].plot(range(ROUNDS), total_energy)
    axs[0, 1].set_title('Total Energy vs Rounds')
    axs[1, 0].plot(range(ROUNDS), throughputs)
    axs[1, 0].set_title('Throughput vs Rounds')
    axs[1, 1].plot(range(ROUNDS), avg_energy)
    axs[1, 1].set_title('Average Energy vs Rounds')
    for ax in axs.flat:
        ax.set(xlabel='Rounds', ylabel='Values')
        ax.grid(True)
    performance_path = 'network_performance.png'
    plt.savefig(performance_path)
    plt.close()
    img = Image.open(performance_path)
    img = img.resize((800, 800), Image.Resampling.LANCZOS)
    img_tk = ImageTk.PhotoImage(img)
    canvas_performance.delete("all")
    canvas_performance.create_image(10, 10, anchor="nw", image=img_tk)
    canvas_performance.image = img_tk
    canvas_performance.config(scrollregion=canvas_performance.bbox("all"))

# -------------- MAIN SIMULASI FUNCTION ---------------
def start_simulation():
    global NUM_NODES, ROUNDS, INITIAL_ENERGY, nodes
    global active_nodes, total_energy, avg_energy, throughputs

    try:
        NUM_NODES = int(entry_nodes.get())
        ROUNDS = int(entry_rounds.get())
        INITIAL_ENERGY = float(entry_energy.get())
        progress_bar.config(maximum=ROUNDS)
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numbers.")
        return

    canvas_topology.delete("all")
    canvas_performance.delete("all")

    reset_death_metrics()
    active_nodes = []
    total_energy = []
    avg_energy = []
    throughputs = []

    if os.path.exists('leach_log.csv'):
        os.remove('leach_log.csv')

    # Inisialisasi posisi node (static)
    if not hasattr(start_simulation, "static_positions") or start_simulation.static_positions is None \
       or len(start_simulation.static_positions) != NUM_NODES:
        positions = []
        for i in range(NUM_NODES):
            x = random.uniform(0, AREA_WIDTH)
            y = random.uniform(0, AREA_HEIGHT)
            positions.append((x, y))
        start_simulation.static_positions = positions
    static_positions = start_simulation.static_positions

    nodes.clear()
    for i in range(NUM_NODES):
        pos = static_positions[i]
        nodes.append(Node(i, pos[0], pos[1], INITIAL_ENERGY))

    for round_num in range(ROUNDS):
        progress_bar['value'] = round_num + 1
        window.update_idletasks()

        # Memperbarui cahaya dan pheromone dengan GSO
        update_glowworm_lighting_and_pheromone()  
        
        # Pemilihan cluster head dengan GSO
        select_cluster_heads_gso(round_num)

        form_clusters()
        throughput = transmit_data()
        check_energy()
        log_round(round_num)
        check_death_metrics(round_num)
        alive_nodes = [node for node in nodes if node.alive]
        active_nodes.append(len(alive_nodes))
        total_energy.append(sum(node.energy for node in nodes))
        avg_energy.append(total_energy[-1] / NUM_NODES if NUM_NODES > 0 else 0)
        throughputs.append(throughput)
        visualize_network(round_num)

    messagebox.showinfo("Simulation Completed", f"Simulation with {NUM_NODES} nodes and {ROUNDS} rounds completed successfully!")
    show_all_topology_images()
    visualize_performance()

# -------------------- GUI LAYOUT STARTS HERE --------------------
window = tk.Tk()
window.title("LEACH Simulation")

window.geometry("600x500")

frame_params = tk.Frame(window)
frame_params.pack(pady=8)

label_nodes = tk.Label(frame_params, text="Number of Nodes:")
label_nodes.grid(row=0, column=0, sticky="w")
entry_nodes = tk.Entry(frame_params, width=8)
entry_nodes.grid(row=0, column=1)

label_rounds = tk.Label(frame_params, text="Number of Rounds:")
label_rounds.grid(row=1, column=0, sticky="w")
entry_rounds = tk.Entry(frame_params, width=8)
entry_rounds.grid(row=1, column=1)

label_energy = tk.Label(frame_params, text="Initial Energy (Joules):")
label_energy.grid(row=2, column=0, sticky="w")
entry_energy = tk.Entry(frame_params, width=8)
entry_energy.grid(row=2, column=1)

button_start = tk.Button(frame_params, text="Start Simulation", command=start_simulation)
button_start.grid(row=3, column=0, columnspan=2, pady=4)

progress_label = tk.Label(window, text="Simulasi berjalan...")
progress_label.pack()
progress_bar = ttk.Progressbar(window, length=280, mode="determinate")
progress_bar.pack(pady=4)

frame_status = tk.Frame(window)
frame_status.pack(pady=2)
label_fnd = tk.Label(frame_status, text="FND: Not achieved")
label_fnd.grid(row=0, column=0, padx=10)
label_hnd = tk.Label(frame_status, text="HND: Not achieved")
label_hnd.grid(row=0, column=1, padx=10)
label_lnd = tk.Label(frame_status, text="LND: Not achieved")
label_lnd.grid(row=0, column=2, padx=10)

# Tabbed Notebook
notebook = ttk.Notebook(window)
notebook.pack(expand=1, fill="both")

tab_topology = tk.Frame(notebook)
notebook.add(tab_topology, text='Network Topology')
canvas_topology = tk.Canvas(tab_topology, width=420, height=420, bg="#f7f7f7")
canvas_topology.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar_y = tk.Scrollbar(tab_topology, orient="vertical", command=canvas_topology.yview)
canvas_topology.configure(yscrollcommand=scrollbar_y.set)
scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)

tab_performance = tk.Frame(notebook)
notebook.add(tab_performance, text='Performance Graph')
canvas_performance = tk.Canvas(tab_performance, width=820, height=820, bg="#f7f7f7")
canvas_performance.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar_performance_y = tk.Scrollbar(tab_performance, orient="vertical", command=canvas_performance.yview)
canvas_performance.configure(yscrollcommand=scrollbar_performance_y.set)
scrollbar_performance_y.pack(side=tk.RIGHT, fill=tk.Y)

nodes = []
window.mainloop()